In [1]:
import numpy as np
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3f14eda362d14bae9e55c7a37853cd8d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='k_CyyL-p6iMlQnJ-DtufqwTR5t0m70hWg8BQMt-toAMG',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3f14eda362d14bae9e55c7a37853cd8d.get_object(Bucket='locationcapstone-donotdelete-pr-zvhzfc05wfadgm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.shape

(103, 3)

Read table from Wikipedia to a dataframe

In [2]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=table[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Get rid of all Not assigned rows

In [3]:
df['Borough'].replace(to_replace='Not assigned',value=np.NaN,inplace=True)
df.dropna(inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
df.shape

(103, 3)

Add latitude and longitude on the data set

In [5]:
df_new=df.join(df_data_1.set_index('Postal Code'),on='Postal Code')

In [6]:
df_new.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Request nearby venue information from Foursquare

In [7]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
CLIENT_ID = '333Z14P1WLW2VUZFQENZSKRKKQAELPQBQSMFLDFJPTYOGK2Z' # your Foursquare ID
CLIENT_SECRET = 'M3HOBBLUC0B3CSV1C3DEVLDAGZ0TK1CFEV5MQ1Q5024APNBZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius=500 
venuelist=[]

for neighborhood_name,lat,lng in zip(df_new['Neighbourhood'],df_new['Latitude'],df_new['Longitude']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for item in results:
        venuelist.append([neighborhood_name,lat,lng,item['venue']['name'],item['venue']['location']['lat'],item['venue']['location']['lng'],item['venue']['categories'][0]['name']])
venuelist

[['Parkwoods',
  43.7532586,
  -79.3296565,
  'Brookbanks Park',
  43.751976046055574,
  -79.33214044722958,
  'Park'],
 ['Parkwoods',
  43.7532586,
  -79.3296565,
  'Variety Store',
  43.75197441585782,
  -79.33311418516017,
  'Food & Drink Shop'],
 ['Parkwoods',
  43.7532586,
  -79.3296565,
  'Corrosion Service Company Limited',
  43.75243208667859,
  -79.33466148402658,
  'Construction & Landscaping'],
 ['Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Victoria Village Arena',
  43.72348055545508,
  -79.31563520925143,
  'Hockey Arena'],
 ['Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Portugril',
  43.72581876267242,
  -79.31278541470671,
  'Portuguese Restaurant'],
 ['Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Tim Hortons',
  43.725516631714754,
  -79.31310251366091,
  'Coffee Shop'],
 ['Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Eglinton Ave E & Sloane Ave/Bermondsey Rd',
  43.726086,
  -79.31362,

In [8]:
venue1=pd.DataFrame(venuelist)
venue1.columns=['Name','Lat','Long','Venue','Venue Lat','Venue Long','Category']
venue1.groupby(['Name']).count()

,Lat,Long,Venue,Venue Lat,Venue Long,Category
Name,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,3,3,3,3,3,3


In [9]:
dummycategory = pd.get_dummies(venue1[['Category']], prefix="", prefix_sep="")
dummycategory['Name']=venue1['Name']
#cols=[[dummycategory.columns[-1]]+list(dummycategory.columns[:-1])]
#dummycategory=dummycategory[cols]
dummycategory

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Mimico NW, The Queensway West, South of Bloor,..."
2100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Mimico NW, The Queensway West, South of Bloor,..."
2101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Mimico NW, The Queensway West, South of Bloor,..."
2102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Mimico NW, The Queensway West, South of Bloor,..."


The mean table used for K means clustering

In [27]:
mean_table=dummycategory.groupby('Name').mean()
mean_table.reset_index(inplace=True)
mean_table

,Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


Create new dataframe with most common venue

In [38]:
mean_table.iloc[1,1:].sort_values(ascending=False).head().index

Index(['Pizza Place', 'Coffee Shop', 'Sandwich Place', 'Pharmacy', 'Pub'], dtype='object')

In [39]:
columns=[]
list1=['st','nd','rd']
for num in range(1,6):
    try:
        columns.append('{}{} most common'.format(num,list1[num-1]))
    except:
        columns.append('{}th most common'.format(num))
df_most_common=pd.DataFrame(columns=columns)
df_most_common['Neighborhood']=mean_table['Name']
df_most_common

,1st most common,2nd most common,3rd most common,4th most common,5th most common,Neighborhood
0,NaN,NaN,NaN,NaN,NaN,Agincourt
1,NaN,NaN,NaN,NaN,NaN,"Alderwood, Long Branch"
2,NaN,NaN,NaN,NaN,NaN,"Bathurst Manor, Wilson Heights, Downsview North"
3,NaN,NaN,NaN,NaN,NaN,Bayview Village
4,NaN,NaN,NaN,NaN,NaN,"Bedford Park, Lawrence Manor East"
...,...,...,...,...,...,...
90,NaN,NaN,NaN,NaN,NaN,"Willowdale, Willowdale East"
91,NaN,NaN,NaN,NaN,NaN,"Willowdale, Willowdale West"
92,NaN,NaN,NaN,NaN,NaN,Woburn
93,NaN,NaN,NaN,NaN,NaN,Woodbine Heights


In [40]:
for index in range(mean_table.shape[0]):
    row=mean_table.iloc[index,1:].sort_values(ascending=False).head().index.values
    df_most_common.iloc[index,0:5]=row
df_most_common

,1st most common,2nd most common,3rd most common,4th most common,5th most common,Neighborhood
0,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Agincourt
1,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Pub,"Alderwood, Long Branch"
2,Coffee Shop,Bank,Deli / Bodega,Ice Cream Shop,Middle Eastern Restaurant,"Bathurst Manor, Wilson Heights, Downsview North"
3,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Bayview Village
4,Coffee Shop,Italian Restaurant,Sandwich Place,Pizza Place,Pharmacy,"Bedford Park, Lawrence Manor East"
...,...,...,...,...,...,...
90,Ramen Restaurant,Café,Pizza Place,Shopping Mall,Sushi Restaurant,"Willowdale, Willowdale East"
91,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Dog Run,"Willowdale, Willowdale West"
92,Coffee Shop,Korean BBQ Restaurant,Drugstore,Diner,Discount Store,Woburn
93,Skating Rink,Spa,Beer Store,Curling Ice,Dance Studio,Woodbine Heights


Run clustering

In [41]:
from sklearn.cluster import KMeans

In [42]:
k_clusters=4
clustering=mean_table.drop(['Name'],axis=1)
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(clustering)
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [43]:
df_most_common.insert(0, 'Cluster Labels', kmeans.labels_)
df_most_common.rename(columns={'Neighborhood':'Neighbourhood'},inplace=True)
df_most_common

,Cluster Labels,1st most common,2nd most common,3rd most common,4th most common,5th most common,Neighbourhood
0,3,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Agincourt
1,3,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Pub,"Alderwood, Long Branch"
2,3,Coffee Shop,Bank,Deli / Bodega,Ice Cream Shop,Middle Eastern Restaurant,"Bathurst Manor, Wilson Heights, Downsview North"
3,3,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Bayview Village
4,3,Coffee Shop,Italian Restaurant,Sandwich Place,Pizza Place,Pharmacy,"Bedford Park, Lawrence Manor East"
...,...,...,...,...,...,...,...
90,3,Ramen Restaurant,Café,Pizza Place,Shopping Mall,Sushi Restaurant,"Willowdale, Willowdale East"
91,3,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Dog Run,"Willowdale, Willowdale West"
92,3,Coffee Shop,Korean BBQ Restaurant,Drugstore,Diner,Discount Store,Woburn
93,3,Skating Rink,Spa,Beer Store,Curling Ice,Dance Studio,Woodbine Heights


In [44]:
df_most_common.set_index('Neighbourhood',inplace=True)
df_most_common.reset_index(inplace=True)
#df_most_common.join(df_new.set_index('Neighbourhood'),on='Neighbourhood')
df_most_common

,Neighbourhood,Cluster Labels,1st most common,2nd most common,3rd most common,4th most common,5th most common
0,Agincourt,3,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio
1,"Alderwood, Long Branch",3,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",3,Coffee Shop,Bank,Deli / Bodega,Ice Cream Shop,Middle Eastern Restaurant
3,Bayview Village,3,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",3,Coffee Shop,Italian Restaurant,Sandwich Place,Pizza Place,Pharmacy
...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",3,Ramen Restaurant,Café,Pizza Place,Shopping Mall,Sushi Restaurant
91,"Willowdale, Willowdale West",3,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Dog Run
92,Woburn,3,Coffee Shop,Korean BBQ Restaurant,Drugstore,Diner,Discount Store
93,Woodbine Heights,3,Skating Rink,Spa,Beer Store,Curling Ice,Dance Studio


## Get the final table with coordinates

In [45]:
df_final=df_most_common.join(df_new.set_index('Neighbourhood'),how='left',on='Neighbourhood')
df_final

,Neighbourhood,Cluster Labels,1st most common,2nd most common,3rd most common,4th most common,5th most common,Postal Code,Borough,Latitude,Longitude
0,Agincourt,3,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",3,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Pub,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",3,Coffee Shop,Bank,Deli / Bodega,Ice Cream Shop,Middle Eastern Restaurant,M3H,North York,43.754328,-79.442259
3,Bayview Village,3,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",3,Coffee Shop,Italian Restaurant,Sandwich Place,Pizza Place,Pharmacy,M5M,North York,43.733283,-79.419750
...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",3,Ramen Restaurant,Café,Pizza Place,Shopping Mall,Sushi Restaurant,M2N,North York,43.770120,-79.408493
91,"Willowdale, Willowdale West",3,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Dog Run,M2R,North York,43.782736,-79.442259
92,Woburn,3,Coffee Shop,Korean BBQ Restaurant,Drugstore,Diner,Discount Store,M1G,Scarborough,43.770992,-79.216917
93,Woodbine Heights,3,Skating Rink,Spa,Beer Store,Curling Ice,Dance Studio,M4C,East York,43.695344,-79.318389


In [46]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [98]:
df_new.loc['Agincourt']

Postal Code            M1S
Borough        Scarborough
Latitude           43.7942
Longitude          -79.262
Name: Agincourt, dtype: object

## Visualize data in a map

In [48]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
latitude=43.6
longitude=-79.38
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighbourhood'], df_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters